In [1]:
#Import necessary packages.

import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM
from keras.models import load_model, Sequential
from keras.optimizers import Adam
import os
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

current_directory = os.getcwd()

/Accounts/huanga2/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load datasets.

X_train = np.load("Training_data.npy")
X_test = np.load("Test_data.npy")
Y_train = np.load('Training_labels.npy')
Y_test = np.load('Test_labels.npy')
test_reference = np.load('Test_reference.npy')
human_train = np.load('Human_Test.npy')
human_test = np.load('Human_Test_Labels.npy')
human_data_reference = np.load('human_Data_Reference.npy')

human_test_dict = {}

for i in range(len(human_data_reference)):
    if human_data_reference[i][0] not in human_test_dict:
        human_test_dict[human_data_reference[i][0]] = [[], []]
    human_test_dict[human_data_reference[i][0]][0].append(human_train[i])
    human_test_dict[human_data_reference[i][0]][1].append(human_test[i])
for item in human_test_dict:
    human_test_dict[item][0] = np.array(human_test_dict[item][0])
    human_test_dict[item][1] = np.array(human_test_dict[item][1])
    
print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))


(469448, 512)
(53080, 512)
469448
53080


In [3]:
#Regularize the data.

mean = X_train.mean(axis = 0)
X_train -= mean
X_test -= mean
for item in human_test_dict:
    human_test_dict[item][0] -= mean 

std = X_train.std(axis = 0)
X_train /= std
X_test /= std
for item in human_test_dict:
    human_test_dict[item][0] /= std

In [10]:
#Group the conjecture-premise pair by name of the conjecture

test_dict = {} 

for i in range(len(X_test)):

    if test_reference[i][0] not in test_dict:
        test_dict[test_reference[i][0]] = [[],[]]
    
    test_dict[test_reference[i][0]][0].append(X_test[i])
    test_dict[test_reference[i][0]][1].append(Y_test[i])




In [125]:
#[OPTIONAL] Choose a smaller sample.

train_data_selection = int(X_train.shape[0]*0.1)
test_data_selection = int(X_test.shape[0]*0.1)

X_train = X_train[:train_data_selection]
Y_train = Y_train[:train_data_selection]
X_test = X_test[:test_data_selection]
Y_test = Y_test[:test_data_selection]

print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))

(47027, 512)
(5225, 512)
47027
5225


In [12]:
#Train network models.
        
for l1 in range(8, 9):
    model = Sequential()
    model.add(Dense(2 ** l1, activation = 'relu', input_shape = (512,)))
    model.add(Dropout(0.5))
    model.add(Dense(2 ** l1, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = Adam(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
        
    history = model.fit(X_train, Y_train, epochs = 1500, batch_size = 4096, shuffle = True,
                                validation_split = 0.1, verbose = 0)
             
    os.chdir('data/model_last')
    model.save('{}x{}_model.h5'.format(2 ** l1, 2 ** l1))
        
    os.chdir('data/model_last')
    with open('{}x{}_model_history.pickle'.format(2 ** l1, 2 ** l1), 'wb') as dictionary:
        pickle.dump(history.history, dictionary, protocol=pickle.HIGHEST_PROTOCOL)
            

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 197,377
Trainable params: 197,377
Non-trainable params: 0
_________________________________________________________________
